Poblem statement: ProcDNA Pharmaceuticals aims to boost prescriptions of "Zederma" for Chronic pain (Topical Analgesic) and "Mercitelo" for Insomnia by deploying a team of sales representatives. Each Zederma Patch costs $2100, and each Mercitelo pill costs $1800. These reps engage physicians through multiple calls to provide comprehensive product information, promoting both products in each outreach. The US is segmented into regions and territories, with reps assigned specific territories for physician engagement. ProcDNA possesses physician prescription data for 2023, detailing total prescriptions per physician for both products and their respective markets. Additionally, the company has prepared a Specialty file outlining targeted specialties for each product which are to be considered for physician outreach.

Question 1:
1. What data checks will you apply on the given datasets?

Solution:
1. Checking for inconsistent data in NPI ID (String size of 10), Zip Code (String size of 5) and missing entries in 01 Prescription Data
2.Check for missing entries in Affiliation file cases:  #NA is present, Account Code is absent, Duplicates in ZTT File
3. Checking for duplicates in all the file

In [1]:
pip install pandas

In [2]:
import pandas as pd

In [3]:
file_path = './data.xlsx'
sheet_name = '1. Prescription Data'
df = pd.read_excel(file_path, sheet_name=sheet_name,header=3)

In [4]:
df.columns

Index(['Unnamed: 0', 'Physician ID', 'Zip Code', 'Zederma TRx',
       'Topical Analgesic market TRx', 'Mercitelo TRx', 'Insomnia Market TRx',
       'Prescription Date'],
      dtype='object')

In [5]:
inconsistent_npi = df[(df['Physician ID'].isna()) | (df['Physician ID'].astype(str).str.len() != 10)]

inconsistent_zip = df[(df['Zip Code'].isna()) | (df['Zip Code'].astype(str).str.len() != 5)]

In [6]:
if not inconsistent_npi.empty:
  print(inconsistent_npi)
else:
  print("NPI ID list is empty")

NPI ID list is empty


In [7]:
if not inconsistent_zip.empty:
  print(inconsistent_zip['Zip Code'].describe())
else:
  print("Zip Code list is empty")

count    5105.000000
mean     5485.476396
std      2576.757813
min       690.000000
25%      2453.000000
50%      6511.000000
75%      7649.000000
max      9180.000000
Name: Zip Code, dtype: float64


In [8]:
print(inconsistent_zip['Zip Code'])

13       2903
14       7444
38       7522
47       1007
48       6614
         ... 
61000    8043
61032    7087
61049    8036
61051    8701
61054    6830
Name: Zip Code, Length: 5105, dtype: int64


In [9]:
# remove the null values
print(inconsistent_zip['Zip Code'].isna().sum()) # no null values

0


In [10]:
# remove the inconsistent data
inconsistent_zip_condition = (df['Zip Code'].isna()) | (df['Zip Code'].astype(str).str.len() != 5)
df_cleaned_zip_code = df[~inconsistent_zip_condition]

In [11]:
# 1.1
print(len(df),len(df_cleaned_zip_code),len(inconsistent_zip))

61070 55965 5105


In [12]:
df_affiliation = pd.read_excel(file_path, sheet_name='4. Affiliation',header=4)

In [13]:
df_affiliation.columns

Index(['Unnamed: 0', 'Physician ID', 'Spec Code', 'Account ID',
       'Account Name'],
      dtype='object')

In [14]:
df_affiliation = df_affiliation.drop(columns=['Unnamed: 0'])

In [15]:
# 1.2
# checking for null values in Affiliation Data Sheet
df_affiliation.isna().sum()

,0
Physician ID,0
Spec Code,0
Account ID,0
Account Name,0


In [16]:
df_ztt = pd.read_excel(file_path, sheet_name='2. ZTT File',header=3)

In [17]:
df_ztt.columns

Index(['Unnamed: 0', 'Zip', 'Territory', 'Region'], dtype='object')

In [18]:
df_ztt = df_ztt.drop(columns=['Unnamed: 0'])
df_ztt.columns

Index(['Zip', 'Territory', 'Region'], dtype='object')

In [19]:
# 1.3 - duplicates in ZTT File
ztt_duplicates = df_ztt[df_ztt.duplicated()]

In [20]:
print(ztt_duplicates)

         Zip       Territory   Region
5806    6074     CT HARTFORD     EAST
5807   10022    NY MANHATTAN    NORTH
5809   91723       CA IRVINE     WEST
5810   71101       TX DALLAS  CENTRAL
5811   28328   NC WILMINGTON     EAST
...      ...             ...      ...
13341  15909   PA PITTSBURGH     EAST
13343  92371    CA RIVERSIDE     WEST
13345  43223   PA PITTSBURGH     EAST
13347  64050  MO KANSAS CITY  CENTRAL
13355  24201    TN KNOXVILLE  CENTRAL

[5234 rows x 3 columns]


In [21]:
print(df_ztt.duplicated().sum())

5234


In [22]:
data_sheets = ['1. Prescription Data','2. ZTT File','3. Speciality File','4. Affiliation']
headers = [3,3,3,4]

In [23]:
for idx,sheet in enumerate(data_sheets):
  df_temp = pd.read_excel(file_path, sheet_name=sheet,header=headers[idx])
  print(sheet)
  print(df_temp[df_temp.duplicated()])

1. Prescription Data
Empty DataFrame
Columns: [Unnamed: 0, Physician ID, Zip Code, Zederma TRx, Topical Analgesic market TRx, Mercitelo TRx, Insomnia Market TRx, Prescription Date]
Index: []
2. ZTT File
       Unnamed: 0    Zip       Territory   Region
5806          NaN   6074     CT HARTFORD     EAST
5807          NaN  10022    NY MANHATTAN    NORTH
5809          NaN  91723       CA IRVINE     WEST
5810          NaN  71101       TX DALLAS  CENTRAL
5811          NaN  28328   NC WILMINGTON     EAST
...           ...    ...             ...      ...
13341         NaN  15909   PA PITTSBURGH     EAST
13343         NaN  92371    CA RIVERSIDE     WEST
13345         NaN  43223   PA PITTSBURGH     EAST
13347         NaN  64050  MO KANSAS CITY  CENTRAL
13355         NaN  24201    TN KNOXVILLE  CENTRAL

[5234 rows x 4 columns]
3. Speciality File
Empty DataFrame
Columns: [Unnamed: 0, Spec Code, Description, Zederma target flag, Mercitelo target flag]
Index: []
4. Affiliation
Empty DataFrame
Column

Problem 2:
Utilize data from Table 1.1 to determine ProcDNA's revenue forecast for both products in 2024. With the total forecast for each product and the Market TRx, calculate Total TRx and $/TRx for each product. Then, segment HCPs into ten deciles based on their Total Market Dollar Potential, including both products, and calculate the count of HCPs in each Decile. Make a visualization for key insights

In [24]:
df_prescription_data = df_cleaned_zip_code
df_speciality_file = pd.read_excel(file_path, sheet_name='3. Speciality File',header=3)
df_affiliation = df_affiliation

In [25]:
df_prescription_affiliation = pd.merge(df_prescription_data, df_affiliation, on='Physician ID', how='inner')

In [26]:
df_final = pd.merge(df_prescription_affiliation, df_speciality_file, left_on='Spec Code', right_on='Spec Code', how='left')

In [27]:
df_final = df_final.fillna(0)

In [28]:
df_final = df_final[['Physician ID','Zederma TRx','Topical Analgesic market TRx','Mercitelo TRx','Insomnia Market TRx','Prescription Date','Spec Code','Zederma target flag','Mercitelo target flag']]
df_final

,Physician ID,Zederma TRx,Topical Analgesic market TRx,Mercitelo TRx,Insomnia Market TRx,Prescription Date,Spec Code,Zederma target flag,Mercitelo target flag
0,7138751703,0.0,972.440012,0.0,0,2023-10-20,AN,1.0,1.0
1,7810991651,0.0,745.830006,0.0,175,2023-10-14,FM,1.0,1.0
2,9485169528,0.0,2713.300032,0.0,26,2023-11-24,PMR,1.0,0.0
3,2168572808,0.0,623.519999,21.0,5121,2023-09-01,N,1.0,1.0
4,4518835174,0.0,1138.690001,0.0,135,2023-12-17,FM,1.0,1.0
...,...,...,...,...,...,...,...,...,...
55960,3810859794,0.0,12.880000,0.0,0,2023-11-18,RHU,1.0,1.0
55961,7025096207,0.0,292.480000,0.0,58,2023-05-10,IM,1.0,1.0
55962,5684024569,0.0,88.720001,8.0,1342,2023-03-12,N,1.0,1.0
55963,2511329025,0.0,398.560002,0.0,0,2023-07-27,RHU,1.0,1.0
